In [ ]:
import os
import sympy
import numpy
import scipy
import scipy.special
from sage.plot.scatter_plot import ScatterPlot

import sympy
import numpy
import scipy
import scipy.special
from sage.plot.scatter_plot import ScatterPlot

def getBoundingCubeHVector(P):
    d = P.dim()
    M = [ 0 for i in range(d)]
    m = [ 0 for i in range(d)]
    for v in P.vertices_list():
        M= [ max(M[i],v[i]) for i in range(d)]
        m= [ min(m[i],v[i]) for i in range(d)]
    cube_ieqs = [ [M[i]] + [ -1 if j==i else 0 for j in range(d) ] for i in range(len(M))]
    cube_ieqs.extend( [ [-m[i]] + [ 1 if j==i else 0 for j in range(d) ] for i in range(len(m))])
    C = Polyhedron(base_ring=ZZ, ieqs = cube_ieqs)
    return getHVectorFromPolytope(C)


def getHVectorFromEhrhartPolynomial(poly):
    t = sympy.symbols("t")
    n= poly.as_poly().degree()+1
    B = [ sympy.binomial(t+n-1-i,n-1) for i in range(n)]
    M = numpy.matrix([[B[j].subs(t,i)  for j in range(len(B))] for i in range(n)], dtype=float)
    b = numpy.array([ poly.subs(t,i) for i in range(n)], dtype=float)
    x = [int(round(i)) for i in numpy.linalg.solve(M,b)]
    return x

def getHVectorFromPolytope(P):
    t = sympy.symbols("t")
    CL = P.ehrhart_polynomial().coefficients(sparse=False)
    pol= sympy.expand(sum([ CL[i]*t^i for i in range(len(CL)) ]))
    return getHVectorFromEhrhartPolynomial( pol)

def createInterpolationPolytope(d,dilation,bounds, cubeHVector, debug=True):

    ie = []
    ie.append([ -bounds[0]  ] + [ int(scipy.special.binom(dilation+d-i,d)) for i in range(d+1)])
    ie.append([ bounds[1] ] + [ -i for i in [ int(scipy.special.binom(dilation+d-i,d)) for i in range(d+1)]])

    # h_0 = 1
    ie.append( [ -1 ] + [ 1 if j==0 else 0 for j in range(d+1)] )
    ie.append( [ 1 ] + [ -1 if j==0 else 0 for j in range(d+1)] )

    # Non negative
    ie.extend( [ [ -1 ] + [ 1 if i==j else 0 for j in range(d+1)] for i in range(1,d+1)])

    # From Beck Stanley et al Thm 3.5
    ie.extend( [ [ 0 ] + [ -1 if 1==j else 1 if j==i else 0 for j in range(d+1)] for i in range(2,d)])
    # From Beck Stanley et al Thm 3.5
    ie.extend( [ [0] + [ 1 if (j<=i+1) else -1 if (j>=d-i) else 0 for j in range(d+1)] for i in range(floor(d/2))]) # is it floor or ceil?
    # From Beck Stanley et al Thm 3.5
    ie.extend( [ [0] + [ -1 if (j<=i) else 1 if (j>=d-i) else 0 for j in range(d+1)] for i in range(floor(d/2))]) # is it floor or ceil?

    # Stanley's monotonicity
    ie.extend([ [cubeHVector[i]] + [ -1 if i==j else 0 for j in range(d+1)] for i in range(d+1)])

    # Stapledon ADDITIVE NUMBER THEORY AND INEQUALITIES IN EHRHART THEORY 1.12
    if d>=3:
        ie.extend( [ [0] + [ -1 if  ((j<=i) and (j>0))  else  1 if ((j>=d-i) and (j<d)) else 0 for j in range(d+1)] for i in range(1,(floor((d-1)/2))+1)])
        ie.extend( [ [0] + [  1 if ((j<=i+1) and (j>1)) else -1 if ((j>=d-i) and (j<d)) else 0 for j in range(d+1)] for i in range(1,(floor((d-1)/2))+1)])

    if debug:
    #if 1:
        print("System: ")
        for i in ie:
            print(i)

    return Polyhedron(ieqs = ie)



def getBounds(P,dilation):
    a = P.ehrhart_polynomial()(dilation)
    return [a,a]

def integerHull(P):
    return Polyhedron(vertices=P.integral_points())

def getStatistics(P, xMin, xMax):

    # NOT SAFE. Check boundary
    #if P.integral_points_count()==len(P.vertices_list()):
    #    raise("Contains no interior LP")
    hvector = getHVectorFromPolytope(P)
    print("h-vector: ", hvector)
    if hvector[-1]==0:
        raise("Contains no interior LP")
    stats ={}
    for dilation in range(xMin,xMax):
        print("--->", dilation)
        #print "dilation:", dilation
        stats[dilation]={}
        S = createInterpolationPolytope(P.dim(),dilation, getBounds(P,dilation), getBoundingCubeHVector(P),debug=False)
        if hvector not in S:
            print(hvector)
            raise Exception("h-vector not in interpolation polytope")
        #print "Interpolation polytope:", S
        stats[dilation]["IP_vertices"] = len(S.vertices_list())
        stats[dilation]["IP_dim"] = S.dim()
        ipc = S.integral_points_count()
        stats[dilation]["IP_lp"] = ipc

        if ipc<2:
            break
#        if true:
#            SIH = integerHull(S)
#            stats[dilation]["hv_in_ih"] = (getHVectorFromPolytope(P) in SIH.vertices_list())
#            stats[dilation]["IH_dim"] =SIH.dim()
#            if ipc==1:
#                break

            #print "Is h-vector a vertex of integer hull? " , (getHVectorFromPolytope(P) in SIH.vertices_list())
    return stats

def createDataFile(P,xMin,xMax,name):
    S = getStatistics(P,xMin,xMax)
    save(S,os.path.join(os.getcwd(), name))

def loadDataFile(name):
    SS= load(os.path.join(os.getcwd(), name))
    return SS


def createGraphFromData(PolyList, Colors, xMin, xMax, yMax):
    G = Graphics()
    for P in PolyList:
        print(P[1])
        SS= load(os.path.join(os.getcwd(), P[1]))
        print(list(zip(range(xMin, xMax), [ SS[s]["IP_lp"] for s in SS])))
        if len(PolyList)>1:
            sp = list_plot(list(zip(range(xMin, xMax), [ SS[s]["IP_lp"] for s in SS])), legend_label=P[1], color=P[2], plotjoined=True, ymax=yMax)
        else:
            sp = list_plot(list(zip(range(xMin, xMax), [ SS[s]["IP_lp"] for s in SS])), legend_label=P[1], color=P[2], plotjoined=True)
        G = G + sp
    return G

def createPlot(PolyList, Colors, xMin, xMax, yMax):
    G = Graphics()
    for P in PolyList:
        print(P[1])
        createDataFile(P[0], xMin, xMax, P[1])
        SS= load(os.path.join(os.getcwd(), P[1]))
        print(zip(range(xMin, xMax), [ SS[s]["IP_lp"] for s in SS]))
        if len(PolyList)>1:
            sp = list_plot(list(zip(range(xMin, xMax), [ SS[s]["IP_lp"] for s in SS])), legend_label=P[1], color=P[2], plotjoined=True, ymax=yMax)
        else:
            sp = list_plot(list(zip(range(xMin, xMax), [ SS[s]["IP_lp"] for s in SS])), legend_label=P[1], color=P[2], plotjoined=True)
        G = G + sp
    return G

def firstDilateWhereHVectorIsExtremal(PolyList):
    D = {}
    for P in PolyList:
        S=loadDataFile(P[1])
        for s in S:
            D[P[1].split("_")[-1]]=-1
            if "hv_in_ih" in  S[s]:
                if S[s]["hv_in_ih"]:
                    D[P[1].split("_")[-1]]=s
                    break
    return D

def firstDilateWhithSingleLP(PolyList):
    D = {}
    for P in PolyList:
        S=loadDataFile(P[1])
        D[P[1].split("_")[-1]]=-1
        for s in S:
            if S[s]["IP_lp"]==1:
                D[P[1].split("_")[-1]]=s
                break
    return D

# L is a dictionary {"family name": PolyList}
def createSummaryAndPlot(L, Colors):
    Summary={}
    for family in L:
        PolyList=L[family]["polylist"]
        #G = createGraphFromData(PolyList, Colors, L[family]["xMin"], L[family]["xMax"], L[family]["yMax"])
        G = createPlot(PolyList, Colors, L[family]["xMin"], L[family]["xMax"], L[family]["yMax"])
        G.save(os.path.join(os.getcwd(),family+".pdf"))
        G.save(os.path.join(os.getcwd(),family+".png"))
        Summary[family]={"firstSingle":firstDilateWhithSingleLP(PolyList), "firstExtremal":firstDilateWhereHVectorIsExtremal(PolyList)}
    save(Summary,os.path.join(os.getcwd(), "summary"))
    return Summary





Colors = ["red","green","blue","orange","black","brown","cyan"]

simplices = [
    [[-3, 1, -5], [2, 4, -4], [3, 3, 3], [4, -3, 0]],
    [[-4, -1, 0], [0, 4, -1], [3, -1, 1], [4, 3, 4]],
    [[-3, -16, 10, 230], [1, -2, -1, 1], [5, 1, -11, -1], [13, 1, -1, -1],[14, 0, 1, -6]],
    [[-4, 0, -1, -1], [1, -8, 0, 2], [1, -6, 1, 1], [2, 1, -2, -7], [5, 0, 5, 0]],
    [[-13, 0, 0, -1, 0], [-3, -1, 4, -1, -5], [0, -2, -20, -1, 1], [0, 1, -7, 0, -6], [1, 1, 2, 6, 0], [10, 1, 1, 2, -6]],
    [[-36, -28, 2, -1, 0], [-33, -1, 4, -1, 0], [-2, 0, 3, 2, 1], [-1, 7, 2, 8, -4], [2, 1, 0, 0, 0], [4, 0, -3, 1, -1]]
]

zonotopes = [
    [[-1, 2, 2], [0, 0, 0], [0, 0, 3], [1, -2, 1], [1, 0, -1], [2, -2, -3],
    [2, -2, 0], [3, -4, -2]],
    [[-3, -1, -2], [-3, -1, 0], [-2, 1, -3], [-2, 1, -1], [1, -3, 0], [1,
    -3, 2], [2, -1, -1], [2, -1, 1]],
    [[-2, 4, -1, -27], [-2, 17, -2, -26], [-1, 0, 0, -24], [-1, 4, -1, -3],
    [-1, 13, -1, -23], [-1, 17, -2, -2], [0, 0, 0, 0], [0, 13, -1, 1], [7,
    4, -1, -28], [7, 17, -2, -27], [8, 0, 0, -25], [8, 4, -1, -4], [8, 13,
    -1, -24], [8, 17, -2, -3], [9, 0, 0, -1], [9, 13, -1, 0]],
    [[-11, 2, 2, 0], [-10, 1, 1, 0], [-10, 1, 4, 0], [-9, 0, 3, 0], [-9, 2,
    -1, -1], [-9, 2, 1, 1], [-8, 1, -2, -1], [-8, 1, 0, 1], [-8, 1, 1, -1],
    [-8, 1, 3, 1], [-7, 0, 0, -1], [-7, 0, 2, 1], [-7, 2, -2, 0], [-6, 1,
    -3, 0], [4, -2, 0, 1], [-5, 0, -1, 0], [-2, 0, 3, 1], [3, -1, 1, 1],
    [-1, -1, 5, 1], [0, -2, 4, 1], [0, 0, 0, 0], [0, 0, 2, 2], [1, -1, -1,
    0], [1, -1, 1, 2], [1, -1, 2, 0], [1, -1, 4, 2], [2, -2, 1, 0], [2, -2,
    3, 2], [2, 0, -1, 1], [3, -1, -2, 1]],
    [[-2, 103, 7, -5, -3], [-2, 104, 5, -4, 1], [-2, 104, 6, -5, -7], [-2,
    105, 4, -4, -3], [-1, 0, 1, -5, 3], [-1, 1, -1, -4, 7], [-1, 1, 0, -5,
    -1], [-1, 2, -2, -4, 3], [-1, 101, 9, -1, -6], [-1, 102, 7, 0, -2], [-1,
    102, 8, -1, -10], [-1, 103, 5, -5, -4], [-1, 103, 6, 0, -6], [-1, 104,
    3, -4, 0], [-1, 104, 4, -5, -8], [-1, 105, 2, -4, -4], [0, -2, 3, -1,
    0], [0, -1, 1, 0, 4], [0, -1, 2, -1, -4], [0, 0, -1, -5, 2], [0, 0, 0,
    0, 0], [0, 1, -3, -4, 6], [0, 1, -2, -5, -2], [0, 2, -4, -4, 2], [0,
    101, 7, -1, -7], [0, 102, 5, 0, -3], [0, 102, 6, -1, -11], [0, 103, 4,
    0, -7], [1, -2, 1, -1, -1], [1, -1, -1, 0, 3], [1, -1, 0, -1, -5], [1,
    0, -2, 0, -1]],
    [[-7, -7, 4, 1, -3], [-7, -6, 2, 1, 0], [-6, -7, 5, 3, -3], [-6, -6, 3,
3, 0], [-5, -8, 3, 2, -6], [-5, -8, 5, -2, -4], [-5, -7, 1, 2, -3], [-5,
-7, 3, -2, -1], [-4, -8, 4, 4, -6], [-4, -8, 6, 0, -4], [-4, -7, 2, 4,
-3], [-4, -7, 4, 0, -1], [-4, -4, 2, -1, -5], [-4, -4, 3, 0, -1], [-4,
-3, 0, -1, -2], [-4, -3, 1, 0, 2], [-3, -9, 4, -1, -7], [-3, -8, 2, -1,
-4], [-3, -4, 3, 1, -5], [-3, -4, 4, 2, -1], [-3, -3, 1, 1, -2], [-3,
-3, 2, 2, 2], [-2, -9, 5, 1, -7], [-2, -8, 3, 1, -4], [-2, -5, 1, 0,
-8], [4, -2, 0, -2, -4], [-2, -5, 3, -4, -6], [-2, -5, 4, -3, -2], [-2,
-4, -1, 0, -5], [-2, -4, 0, 1, -1], [-2, -4, 1, -4, -3], [-2, -4, 2, -3,
1], [-1, -5, 2, 2, -8], [-1, -5, 3, 3, -4], [-1, -5, 4, -2, -6], [-1,
-5, 5, -1, -2], [-1, -4, 0, 2, -5], [-1, -4, 1, 3, -1], [4, -3, 2, -2,
-7], [-1, -4, 3, -1, 1], [-1, -1, 1, -2, -3], [-1, 0, -1, -2, 0], [0,
-6, 2, -3, -9], [0, -6, 3, -2, -5], [0, -5, 0, -3, -6], [0, -5, 1, -2,
-2], [0, -1, 2, 0, -3], [0, 0, 0, 0, 0], [1, -6, 3, -1, -9], [1, -6, 4,
0, -5], [1, -5, 1, -1, -6], [1, -5, 2, 0, -2], [1, -2, 0, -1, -6], [1,
-2, 2, -5, -4], [1, -1, -2, -1, -3], [1, -1, 0, -5, -1], [2, -2, 1, 1,
-6], [2, -2, 3, -3, -4], [2, -1, -1, 1, -3], [2, -1, 1, -3, -1], [3, -3,
1, -4, -7], [3, -2, -1, -4, -4]]
]

L = {
        #  "cross_polytope" : {
        #     "polylist": [ [ polytopes.cross_polytope(i), "cross_polytope_"+str(i), Colors[i-3]] for i in range(3,7) ],
        #     "yMax": 100,
        #     "xMin": 2,
        #     "xMax": 100
        #  }
        # "solids": {
        #     "polylist": [
        #                     [polytopes.octahedron(), "octahedron", Colors[0]],
        #                     [polytopes.cuboctahedron(), "cuboctahedron", Colors[1]],
        #                     [polytopes.rhombic_dodecahedron(), "rhombic_dodecahedron", Colors[2]],
        #                     [polytopes.truncated_octahedron(), "truncated_octahedron", Colors[3]]
        #                     #[polytopes.Kirkman_icosahedron(), "Kirkman_icosahedron", Colors[4]]
        #                ],
        #     "yMax": 50,
        #     "xMin": 2,
        #     "xMax": 40
        # }
        # "cubes": {
        #    "polylist":  [ [ polytopes.hypercube(i), "cube_"+str(i), Colors[i-3]] for i in range(3,7) ],
        #        "yMax": 100,
        #        "xMin": 3,
        #        "xMax": 100
        # }#,
        # "permutahedra4": {
        #    "polylist": [ [ polytopes.permutahedron(i), "permutahedron"+str(i), Colors[i-4]] for i in range(4,5) ],
        #    "yMax" :300,
        #    "xMin": 5,
        #    "xMax": 100
        # },
        # "permutahedra5": {
        #     "polylist": [ [ polytopes.permutahedron(i), "permutahedron"+str(i), Colors[i-5]] for i in range(5,6) ],
        #     "yMax" :1000,
        #     "xMin": 2,
        #     "xMax": 101
        # },
        # "permutahedra5_b": {
        #     "polylist": [ [ polytopes.permutahedron(i), "permutahedron"+str(i), Colors[i-5]] for i in range(5,6) ],
        #     "yMax" :1000,
        #     "xMin": 101,
        #     "xMax": 201
        # },
        # "permutahedra5_c": {
        #     "polylist": [ [ polytopes.permutahedron(i), "permutahedron"+str(i), Colors[i-5]] for i in range(5,6) ],
        #     "yMax" :1000,
        #     "xMin": 201,
        #     "xMax": 301
        # },
        # "permutahedra5_d": {
        #     "polylist": [ [ polytopes.permutahedron(i), "permutahedron"+str(i), Colors[i-5]] for i in range(5,6) ],
        #     "yMax" :1000,
        #     "xMin": 301,
        #     "xMax": 401
        # },
        # "permutahedra5_e": {
        #     "polylist": [ [ polytopes.permutahedron(i), "permutahedron"+str(i), Colors[i-5]] for i in range(5,6) ],
        #     "yMax" :1000,
        #     "xMin": 401,
        #     "xMax": 601
        # },
        # "permutahedra5_f": {
        #     "polylist": [ [ polytopes.permutahedron(i), "permutahedron"+str(i), Colors[i-5]] for i in range(5,6) ],
        #     "yMax" :1000,
        #     "xMin": 601,
        #     "xMax": 801
        # },
        # "permutahedra5_g": {
        #     "polylist": [ [ polytopes.permutahedron(i), "permutahedron"+str(i), Colors[i-5]] for i in range(5,6) ],
        #     "yMax" :1000,
        #     "xMin": 801,
        #     "xMax": 1001
        # },
        # "permutahedra5_h": {
        #     "polylist": [ [ polytopes.permutahedron(i), "permutahedron"+str(i), Colors[i-5]] for i in range(5,6) ],
        #     "yMax" :1000,
        #     "xMin": 1001,
        #     "xMax": 1201
        # },
        # "permutahedra5_i": {
        #     "polylist": [ [ polytopes.permutahedron(i), "permutahedron"+str(i), Colors[i-5]] for i in range(5,6) ],
        #     "yMax" :1000,
        #     "xMin": 1201,
        #     "xMax": 1301
        # }#,
        # "permutahedra6": {
        #    "polylist": [ [ polytopes.permutahedron(i), "permutahedron"+str(i), Colors[i-6]] for i in range(6,7) ],
        #    "yMax" :1000,
        #    "xMin": 5,
        #    "xMax": 100
        # },
    #     "simplices" : {
    #        "polylist": [ [ Polyhedron(vertices=simplices[i]), "simplex"+str(i), Colors[i]] for i in range(6) ],
    #        "yMax" :75,
    #        "xMin": 20,
    #        "xMax": 50
    #    },
       "zonotopes" : {
          "polylist": [ [ Polyhedron(vertices=zonotopes[i]), "zonotope"+str(i), Colors[i]] for i in range(5) ],
          "yMax" :40,
          "xMin": 30,
          "xMax": 100
      }
    }

print(createSummaryAndPlot(L, Colors))